In [0]:
import torch
import torchtext 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

In [2]:
#Load data
TEXT = torchtext.data.Field(lower=True, include_lengths=True, batch_first=True)
LABEL = torchtext.data.Field(sequential=False)

train,test = torchtext.datasets.IMDB.splits(TEXT, LABEL)

print("Total no. of examples in training set :", len(train))
print("Total no. of examples in test set :", len(test))
print("Lables : ", set(train.label))

Total no. of examples in training set : 25000
Total no. of examples in test set : 25000
Lables :  {'neg', 'pos'}


In [3]:
#check gpu
if torch.cuda.is_available():
    cuda0 = torch.device("cuda:0") 
    print("Running on the GPU")
else:
    cuda0 = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


### Using pre-trained embedding (Glove)

In [0]:
from torchtext.vocab import GloVe  

In [0]:
#initialize glove embeddings
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=200)) # This will download 862 MB of pre-trained vectors 
LABEL.build_vocab(train)

In [0]:
class LSTMSentimentAnalysis(nn.Module):
  def __init__(self, vocab_size ,embedding_dim, hidden_dim1 ,no_of_layers2,output_dim, bidirectional, dropout):
    super().__init__()
    self.embedding = nn.Embedding(num_embeddings=vocab_size, 
                                  embedding_dim=embedding_dim)
    self.lstm1 = nn.LSTM(input_size= embedding_dim, 
                        hidden_size= hidden_dim1, 
                        num_layers= no_of_layers1, 
                        batch_first=True, 
                        bidirectional = bidirectional,
                        dropout=dropout) 
    self.fc1 = nn.Linear(hidden_dim1, 16)
    self.fc2 = nn.Linear(16, output_dim)


  def forward(self,text,text_length):
    #text = [batch size,sent_length]
    embedded = self.embedding(text)
    #embedded = [batch_size, sent_len, emb_dim]

    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded,
                                                        lengths=text_length, 
                                                        batch_first=True)

    packed_output, (hidden, cell) = self.lstm1(packed_embedded)
    
    hidden = hidden [-2,:,:]

    out = F.relu(self.fc1(hidden))
    out = self.fc2(out)

    return out


In [0]:
#Hyperparameters
vocab_size = len(TEXT.vocab)
embedding_dim = 128
hidden_dim1 = 32
no_of_layers1 = 4
output_dim = 2
bidirectional = False
dropout = 0.2
no_of_epoches = 35
batch_s = 512


In [0]:
#define model
model = LSTMSentimentAnalysis(vocab_size ,embedding_dim, hidden_dim1,no_of_layers1, output_dim, bidirectional, dropout)
#define optimizer
optimizer = optim.Adam(model.parameters())
#define loss 
loss_function = nn.CrossEntropyLoss()

In [18]:
print(model)

LSTMSentimentAnalysis(
  (embedding): Embedding(251639, 128)
  (lstm1): LSTM(128, 32, num_layers=4, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=32, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=2, bias=True)
)


In [19]:
#transfer model to gpu
model.to(cuda0)
loss_function.to(cuda0)

CrossEntropyLoss()

In [0]:
# Training the network
def train_Lstm(model):
  for epoch in range(no_of_epoches):
    i=0
    loss_per_epoch = 0.0
    model.train(mode=True)
    for batch in train_iter:
      i+= 1
      text, text_length = batch.text
      output = model(text,text_length).squeeze() # convert 1d tensor
      batch.label = batch.label -1
      loss = loss_function(output, batch.label)
      loss.backward() 
      optimizer.step()

      loss_per_epoch+= loss.item()

    print("EPOCH : {} | Loss : {}".format(epoch+1,loss_per_epoch/i))

In [0]:
#Testing the network
def evaluate_model(model):
  predicted = torch.tensor([0], device=cuda0)
  true_classes = torch.tensor([0], device= cuda0)
  model.eval() # model in evaluate mode
  for batch in test_iter:
    text, text_length = batch.text
    output = model(text,text_length).squeeze() 
    _,pred = torch.max(output,1)
    predicted = torch.cat([predicted,pred],0)
    true_classes = torch.cat([true_classes,batch.label],0)
  
  predicted = predicted[1:]
  true_classes = true_classes[1:]
  return predicted, true_classes

def accuracy(true_label,predicted):
  predicted = predicted+1
  dic ={1:0,2:0}
  count=0
  for i in range(len(predicted)):
    if predicted[i]==true_label[i]:
      count+=1
      if predicted[i].item()==1:
        dic[1]+=1
      else:
        dic[2]+=1

  dic[1]=dic[1]/12500
  dic[2]=dic[2]/12500
  acc= (count*100)/len(predicted)
  print("Total Accuracy {}% :", acc)
  print("Accuracy per class, Positive : {}% | Negative : {}%".format(dic[1]*100,dic[2]*100))
  



In [22]:
#Converting input data into batches
train_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train, test), batch_size=batch_s, sort_key = lambda x: len(x.text),
    sort_within_batch=True,device=cuda0) 

# Start training
train_Lstm(model)

EPOCH : 1 | Loss : 0.6913803207631014
EPOCH : 2 | Loss : 0.6477658845940415
EPOCH : 3 | Loss : 0.6564758602453737
EPOCH : 4 | Loss : 0.6830433509787734
EPOCH : 5 | Loss : 0.5764267128341052
EPOCH : 6 | Loss : 0.5266346116455234
EPOCH : 7 | Loss : 0.48329331923504265
EPOCH : 8 | Loss : 0.43350294870989664
EPOCH : 9 | Loss : 0.41658115234910226
EPOCH : 10 | Loss : 0.342253924632559
EPOCH : 11 | Loss : 0.2940977446886958
EPOCH : 12 | Loss : 0.22026983724564922
EPOCH : 13 | Loss : 0.19451882264443807
EPOCH : 14 | Loss : 0.17763513995676625
EPOCH : 15 | Loss : 0.1336038029011415
EPOCH : 16 | Loss : 0.10936599817811227
EPOCH : 17 | Loss : 0.07923401930198377
EPOCH : 18 | Loss : 0.08430692355851738
EPOCH : 19 | Loss : 0.06326940222358217
EPOCH : 20 | Loss : 0.0823183325784547
EPOCH : 21 | Loss : 0.0512175767169315
EPOCH : 22 | Loss : 0.06757436715522591
EPOCH : 23 | Loss : 0.04534472403477649
EPOCH : 24 | Loss : 0.05769603452360143
EPOCH : 25 | Loss : 0.047000702242462
EPOCH : 26 | Loss : 0.0

In [23]:
#Start Testing
predicted , true_label = evaluate_model(model)
accuracy(true_label,predicted)

Total Accuracy {}% : 77.192
Accuracy per class, Positive : 75.24% | Negative : 79.144%


In [0]:
#Save the model
PATH = './sentiment_analysis.pth'
torch.save(model.state_dict(), PATH)

In [26]:
#load network
net_loaded = LSTMSentimentAnalysis(vocab_size ,embedding_dim, hidden_dim1,no_of_layers1, output_dim, bidirectional, dropout)
net_loaded.load_state_dict(torch.load(PATH))
print("Model Loaded sucessfully.")

Model Loaded sucessfully.
